# Lifting bodies

This section will modify the vortex panel method to enable the prediction of lift on wing sections.

## Foil section geometry

To find the lift on a foil, we first need to define the geometry. Let's use a symmetric Jukowski foil for now. This foil shape is obtained by applying the following mapping

$$ x' = \frac x2 (1+1/r^2), \quad y' = \frac y2 (1-1/r^2),\quad r^2= x^2+y^2 $$

to the points on a circle which intersects $x,y=1,0$. I've followed this procedure in the function `VortexPanel.make_jfoil`. Let's import that module as `vp` again:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import VortexPanel as vp
help(vp.make_jfoil)

The arguments are the number of points $N$ and the center of the circle before mapping. Let's test it out.

In [ ]:
foil = vp.make_jfoil(N=32)
foil.plot()
plt.axis('equal');

##### Quiz

How can you get a cambered (ie, curved) foil shape using the Jukowski transform?

1. Shift the center right or left.
1. Shift the center up or down.
1. Change the circle's radius.
---

## Flows with circulation

Now lets solve for the flow around the foil at an angle of attack $\alpha$:

In [ ]:
alpha = np.pi/16         # set angle of attack
foil.solve_gamma(alpha)  # solve for gamma
foil.plot_flow()         # plot the flow

Sigh. There are so many things wrong with this picture...

 - The flow is faster on the underside of the foil. 
 - The rear stagnation point is on the upper side of the foil.
 - There is a singularity on the lower side of the trailing edge.

These are all physically incorrect! But how can setting the (physical) no-slip condition on each panel lead to non-physical results?

##### Mathematics fundamental: Necessary but insufficient
##### The no-slip condition is not sufficient to uniquely determine the flow around lifting bodies

This means there are many (non-physical) solutions for this flow, one of which is given above.

## The Kutta condition

We need an additional condition to determine the physical solution for this flow.

##### Quiz

What does the flow need to look like on the trailing edge?

![short](resources\trailingEdge.png)

1. The flow needs to wrap from the bottom to the top.
1. The flow needs to wrap from the top to the bottom.
1. The flow needs to separate from the edge. 

---
The fundamental problem with the solution for the foil above is that the flow is wrapping around the sharp trailing edge. This means that particles traveling on the body streamline *instantly* change direction - which is impossible for any object with mass.

Avoiding this impossibility is called the Kutta condition.

##### Hydrodynamics fundamental: Kutta condition
##### "Potential flow must separate from a sharp trailing edge"

Look at the sketch above, we can see $\gamma$ is antisymmetric about the trailing edge, ie $\gamma(\epsilon)+\gamma(-\epsilon)=0$. This additional condition uniquely determines the correct solution $\gamma_i$. 

---

The Kutta condition has already been added to the solver. You may have noticed that the `solve_gamma` function takes an optional `kutta` argument. This lets you define any number trailing edges (by a list of index pairs) and enforce the Kutta condition on them all.

The trailing edge of the Jukowski foil is bordered by the first and last panels; ie, we want `kutta=[(0,N-1)]`. Lets, try it out!

In [ ]:
foil = vp.make_jfoil(N=32)               # make the geom
alpha = np.pi/16                         # set angle of attack
foil.solve_gamma(alpha,kutta=[(0,-1)])   # solve
foil.plot_flow()                         # plot

The flow plot shows the correct qualitative features:
- High speed flow on the top.
- Slow flow on the trailing edge.
- As $\alpha$ is increased, the amount of circulation increases.

To test if this function really does apply the Kutta condition, let's plot $s$ versus $\gamma$, as in the last notebook:

In [ ]:
alpha = np.pi/16                     # angle of attack
foil = vp.make_jfoil(N=32)           # make geometry
s = foil.distance()                  # distance array

# solve without kutta and plot 
foil.solve_gamma(alpha)                 # solve
gamma = foil.get_array('gamma')         # get
plt.plot(s,gamma,label="No kutta")      # plot

# solve with kutta and plot
foil.solve_gamma(alpha,kutta=[(0,-1)])  # solve
gamma = foil.get_array('gamma')         # get
plt.plot(s,gamma,label="With kutta")    # plot

# finish gamma(s) plot
plt.legend()
plt.xlabel(r'$s$', fontsize=20)
plt.ylabel(r'$\tilde\gamma$', fontsize=20, rotation=0)
plt.axhline(0,c='k',ls='--')
plt.show()

The Kutta condition __has__ been successfully enforced since $\gamma(0)=-\gamma(-1)$.

---

## Lift force

The lift force on any body in 2D potential flow is given by the total circulation:

$$ L = -\rho U \Gamma = -\rho U^2 \tilde\Gamma $$

where $\tilde\Gamma=\oint\tilde\gamma(s)ds$. Note that since we solve with $|U|=1$ our $\gamma = \tilde\gamma$.

We non-dimensionalize the lift as the lift coefficient

$$ C_L =\frac L{\tfrac 12 \rho U^2 c} $$

where $c$ is the coord length, the distance from the leading to trailing edge. Note that the vortex panel


---

##### Quiz

What is the expression for the lift coefficient in terms `gamma` and the panel half-width `S`?
- `-sum(gamma)/(0.5*rho*c*U)`
- `-sum(gamma*2*S)/(0.5*c)`
- `-sum(gamma)*2*S/0.5*c*U`
---

Copy this code into the function below to measure the lift on our foil

In [ ]:
def C_L(panels):
    # panel attribute arrays
    gamma, xc, S = panels.get_array('gamma','xc','S')

    # chord length
    c = max(xc)-min(xc)    
    
    # return the lift coefficient
    
print('The C_L={:.3g}'.format(C_L(foil)))

##### Numerical fundamental: Validation
##### Every piece of code must be tested against a known nontrivial example

The reason I implemented a Jukowski foil is not because it is a great foil shape (it isn't). It is because there is an analytic solution which we can use for __validation__

$$C_L = 2\pi \left(1+\frac 4{3\sqrt 3} \frac tc \right)\sin\alpha$$

where $t/c$ is the thickness to coord ratio.

##### Your turn

 - ** Complete ** the function `solve_C_L` to compute $C_L$ for a `PanelArray` given an angle of attack. 
 - ** Plot and Compare ** the numerical C_L for $\alpha=0,\ldots,15^o$ against the analytic solution.
 - ** Consider ** the results. Do they converge? Are they validated? 
 - ** Finally ** Do they indicate the point of *stall*? Why or why not?

In [ ]:
def solve_C_L(panels,alpha,kutta=[(0,-1)]):
    # your code here
    # return C_L

# Test it here. Does it give the same number as above? Does the value change with alpha?...

In [ ]:
def analytic_C_L(alpha,t_c=0.2355): # t/c of jfoil xcen=-0.1
    return 2*np.pi*(1+t_c*4/(3*np.sqrt(3)))*np.sin(alpha)

alpha_deg = np.linspace(0,15,6)
alpha = alpha_deg*np.pi/180.
plt.plot(alpha_deg,analytic_C_L(alpha), 'k', label='analytic')

for N in 2**np.arange(7,2,-1):
    foil = vp.make_jfoil(N=N)
#     CL = [solve_C_L(foil,a) for a in alpha]
#     plt.plot(alpha_deg, CL, '--',label=N)

plt.legend(loc='lower right')
plt.xlabel(r'$\alpha$ (deg)', fontsize=15)
plt.ylabel(r'$C_L$', fontsize=15, rotation=0)
plt.yticks([0,1,2]);

### Linear panels

Up to this point, we've used panels with constant strength because they are simple and effective. Extending the method to use panels with linearly varying strength is only slightly more complex and much more accurate.

![medium](resources\panel_O2.png)

We don't need to consider the detailed derivation (it is similar to the constant strength panels but with more terms). The important point is that the velocity function for a linearly varying strength panel can be written as

$$\vec u_i(x,y) = \gamma_{i-1}(S)\ \vec f_i(x,y,-S)+\gamma_i(S)\ \vec f_i(x,y,S)$$

where $\gamma_i(S)$ is the strength on the positive end of panel $i$, and $\vec f_i(x,y,\pm S)$ are two new influence functions. Just as with the constant-strength panel, the velocity depends __linearly__ on $\gamma$, and for $N$ panels we have only $N$ unknowns. Therefore, we can construct an $N$ by $N$ linear system.

---


##### Quiz

Approximately how many constant strength panels will we need to match the accuracy of $N$ linearly varying panels?

1. $N$
1. $2N$
1. $N^2$

---

Find the answer yourself by using second order panels in the foil test above. Simply call `PanelArray.solve_gamma_O2` (thats a letter O, not a zero) instead of `solve_gamma`. This function has the same input arguments and the resulting panel array still works with `plot_flow`, `get_array`, etc.

As you can see, the panels with linearly varying strength give a __much__ better result for a given $N$. This is because the error between the true analytic $\gamma(s)$ and a piecewise constant approximation is $\epsilon\sim 1/N$, whereas it is $\epsilon\sim (1/N)^2$ for a piecewise linear approximation. Because of the exponent, this is called _second-order_ or _O(2)_ error. 

This is worth saying again: We are __massively__ reducing the error without solving __any__ extra equations!

##### Numerical fundamental: O(2)
##### Second-order methods are unreasonably effective

This improvement is fairly general and second-order methods (or greater) should be used whenever possible. For example, the trapezoidal rule is a second order method.